In [5]:
import torch
import pandas as pd
from pprint import pprint
from sklearn.model_selection import train_test_split
from unidecode import unidecode
from collections import Counter
import nltk
from nltk import word_tokenize
nltk.download('punkt')
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import numpy as np

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sures\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


In [6]:
movies = pd.read_csv("movie.csv")
df=movies.copy()
df.head(10)

,text,label
0,I grew up (b. 1965) watching and loving the Th...,0
1,"When I put this movie in my DVD player, and sa...",0
2,Why do people who do not know what a particula...,0
3,Even though I have great interest in Biblical ...,0
4,Im a die hard Dads Army fan and nothing will e...,1
5,A terrible movie as everyone has said. What ma...,0
6,Finally watched this shocking movie last night...,1
7,I caught this film on AZN on cable. It sounded...,0
8,It may be the remake of 1987 Autumn's Tale aft...,1
9,My Super Ex Girlfriend turned out to be a plea...,1


In [8]:
df = pd.read_csv('IMDB-Dataset.csv')
#df = df[:10000]
df.sentiment = df.sentiment.apply(lambda x: 1 if x=='positive' else 0)
len(df)

50000

In [9]:
import re
import html
from bs4 import BeautifulSoup

def lower(review):
    return review.lower()

def deletePunctuation(x):
    return re.sub(r'[^\w\s]', ' ', x)

def deleteSpaces(x):
    return re.sub(r'\s+', ' ', x)

def unescape(x):
    return html.unescape(x)

def deleteHtmlTags(x):
    return BeautifulSoup(x, "lxml").text

print(df[:10])
df.review = df.review.apply(lower)
df.review = df.review.apply(deleteHtmlTags)
df.review = df.review.apply(unidecode)
df.review = df.review.apply(deletePunctuation)
df.review = df.review.apply(unescape)
df.review = df.review.apply(deleteSpaces)
print(df[:10])

                                              review  sentiment
0  One of the other reviewers has mentioned that ...          1
1  A wonderful little production. <br /><br />The...          1
2  I thought this was a wonderful way to spend ti...          1
3  Basically there's a family where a little boy ...          0
4  Petter Mattei's "Love in the Time of Money" is...          1
5  Probably my all-time favorite movie, a story o...          1
6  I sure would like to see a resurrection of a u...          1
7  This show was an amazing, fresh & innovative i...          0
8  Encouraged by the positive comments about this...          0
9  If you like original gut wrenching laughter yo...          1
                                              review  sentiment
0  one of the other reviewers has mentioned that ...          1
1  a wonderful little production the filming tech...          1
2  i thought this was a wonderful way to spend ti...          1
3  basically there s a family where a li

In [10]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")
from nltk.corpus import stopwords
stop = stopwords.words('english')

df["stemmed"] = df.review.apply(lambda x: [stemmer.stem(y) for y in x.split() if stemmer.stem(y) not in (stop)])
df

,review,sentiment,stemmed
0,one of the other reviewers has mentioned that ...,1,"[one, review, mention, watch, 1, oz, episod, h..."
1,a wonderful little production the filming tech...,1,"[wonder, littl, product, film, techniqu, veri,..."
2,i thought this was a wonderful way to spend ti...,1,"[thought, wonder, way, spend, time, hot, summe..."
3,basically there s a family where a little boy ...,0,"[basic, famili, littl, boy, jake, think, zombi..."
4,petter mattei s love in the time of money is a...,1,"[petter, mattei, love, time, money, visual, st..."
...,...,...,...
49995,i thought this movie did a down right good job...,1,"[thought, movi, right, good, job, creativ, ori..."
49996,bad plot bad dialogue bad acting idiotic direc...,0,"[bad, plot, bad, dialogu, bad, act, idiot, dir..."
49997,i am a catholic taught in parochial elementary...,0,"[cathol, taught, parochi, elementari, school, ..."
49998,i m going to have to disagree with the previou...,0,"[go, disagre, previous, comment, side, maltin,..."


In [11]:
pd.value_counts(np.hstack(df.stemmed))

movi       103216
film        95837
one         55427
like        45187
time        31945
            ...  
possibi         1
sme             1
235th           1
tvcat           1
movie8          1
Length: 72969, dtype: int64

In [12]:
noOfWords = 10000
vocab_df = pd.value_counts(np.hstack(df.stemmed)).keys()
vocabulary = list(vocab_df[:noOfWords])
print(vocabulary[:20])

['movi', 'film', 'one', 'like', 'time', 'good', 'make', 'charact', 'see', 'get', 'watch', 'veri', 'even', 'stori', 'would', 'onli', 'realli', 'well', 'scene', 'look']


In [13]:
token_indices = dict((t, i + 2) for i, t in enumerate(vocabulary))
indices_token = dict((i + 2, t) for i, t in enumerate(vocabulary))

indices_token[0] = 'UNK'
token_indices['UNK'] = 0

indices_token[1] = 'PAD'
token_indices['PAD'] = 1

print('Token Indices Dimension:', len(indices_token))

Token Indices Dimension: 10002


In [75]:
train_df, test_df = train_test_split(df, test_size=0.1, random_state = 42)

print('Test Data:', len(test_df))
train_df, valid_df = train_test_split(train_df, test_size=(0.1/0.9), random_state = 42)

print('Train Data:', len(train_df))
print('Validation Data:', len(valid_df))
test_df.head(5)

Test Data: 5000
Train Data: 40000
Validation Data: 5000


,review,sentiment,stemmed
33553,i really liked this summerslam due to the look...,1,"[realli, like, summerslam, due, look, arena, c..."
9427,not many television shows appeal to quite as m...,1,"[mani, televis, show, appeal, quit, mani, diff..."
199,the film quickly gets to a major chase scene w...,0,"[film, quick, get, major, chase, scene, ever, ..."
12447,jane austen would definitely approve of this o...,1,"[jane, austen, would, definit, approv, one, gw..."
39489,expectations were somewhat high for me when i ...,0,"[expect, somewhat, high, went, see, movi, thou..."


In [15]:
def vectorize_sentences(data, token_indices, one_hot = False):
    vectorized = []
    for sentences in data:

        
        sentences_of_indices = [token_indices[w] if w in token_indices.keys() else token_indices['UNK'] for w in sentences]

        
        if one_hot:
            sentences_of_indices = np.eye(len(token_indices))[sentences_of_indices]

        vectorized.append(sentences_of_indices)

    return vectorized

train_reviews_vectorized = vectorize_sentences(train_df.stemmed, token_indices)
test_reviews_vectorized = vectorize_sentences(test_df.stemmed, token_indices)
valid_reviews_vectorized = vectorize_sentences(valid_df.stemmed, token_indices)

In [16]:
vectors_dim = [len(repr) for repr in train_reviews_vectorized]
vectors_dim[:5]

[63, 173, 54, 125, 83]

In [17]:
def pad(samples, max_length):
    
    return torch.tensor([
        sample[:max_length] + [1] * max(0, max_length - len(sample))
        for sample in samples
    ])

In [18]:
max_len = 800
train_reviews_vectorized = pad(train_reviews_vectorized, max_length = max_len)
test_reviews_vectorized = pad(test_reviews_vectorized, max_length = max_len)
valid_reviews_vectorized = pad(valid_reviews_vectorized, max_length = max_len)

In [19]:
train_reviews_vectorized

tensor([[ 192,   27,   33,  ...,    1,    1,    1],
        [ 570,    2,  423,  ...,    1,    1,    1],
        [   2,   13,    7,  ...,    1,    1,    1],
        ...,
        [  71,   18,   28,  ...,    1,    1,    1],
        [   2,  902,   23,  ...,    1,    1,    1],
        [ 214, 2689,  141,  ...,    1,    1,    1]])

In [20]:
vectors_dim = [len(repr) for repr in train_reviews_vectorized]
vectors_dim[:5]

[800, 800, 800, 800, 800]

In [21]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, samples, labels):
        self.samples = samples
        self.labels = labels
            
    def __getitem__(self, k):
        
        return self.samples[k], self.labels[k]
    
    def __len__(self):
        
        return len(self.samples)

In [24]:
class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.embedding = torch.nn.Embedding(len(indices_token), 100, padding_idx=1)
        
        conv1 = torch.nn.Sequential(
            torch.nn.Conv1d(in_channels=100, out_channels=128, kernel_size=3, padding=1),
            torch.nn.BatchNorm1d(128),
            torch.nn.ReLU(),
            torch.nn.MaxPool1d(kernel_size=2)
        )
        conv2 = torch.nn.Sequential(
            torch.nn.Conv1d(in_channels=128, out_channels=128, kernel_size=5, padding=2),
            torch.nn.BatchNorm1d(128),
            torch.nn.ReLU(),
            torch.nn.MaxPool1d(kernel_size=2)
        )
        conv3 = torch.nn.Sequential(
            torch.nn.Conv1d(in_channels=128, out_channels=128, kernel_size=5, padding=2),
            torch.nn.BatchNorm1d(128),
            torch.nn.ReLU(),
            torch.nn.MaxPool1d(kernel_size=2)
        )
        
        # 800 / 8
        global_average = torch.nn.AvgPool1d(kernel_size=100, stride=100)

        self.convolutions = torch.nn.Sequential(
            torch.nn.Dropout(p=0.4),
            conv1, conv2, conv3, global_average
        )

        flatten = torch.nn.Flatten()
        linear = torch.nn.Linear(in_features=128, out_features=2)
        self.classifier = torch.nn.Sequential(flatten, linear)
        
    def forward(self, input):
        embeddings = self.embedding(input)
        embeddings = embeddings.permute(0, 2, 1)
        output = self.convolutions(embeddings)
        output = self.classifier(output)
        return output

In [84]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# instanțiem modelul
model = Model().to(DEVICE)

# Adam optimizer cu lr = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Cross Entropy loss
loss_fn = torch.nn.CrossEntropyLoss()

# training dataset and dataloader
# test dataset and dataloader
train_ds = Dataset(train_reviews_vectorized, train_df.sentiment.tolist())
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=64, shuffle=True)
test_ds = Dataset(test_reviews_vectorized, test_df.sentiment.tolist())
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=64, shuffle=False)
valid_ds = Dataset(valid_reviews_vectorized, valid_df.sentiment.tolist())
valid_dl = torch.utils.data.DataLoader(valid_ds, batch_size=64, shuffle=False)
print(len(test_df.sentiment.tolist()))

5000


In [65]:
best_val_acc = 0
for epoch_n in range(10):
    print(f"Epoch #{epoch_n + 1}")
    model.train()
    for batch in train_dl:
        model.zero_grad()

        inputs, targets = batch
        inputs = inputs.long().to(DEVICE)
        targets = targets.to(DEVICE)

        output = model(inputs)
        loss = loss_fn(output, targets)

        loss.backward()
        optimizer.step()

    mean_loss = 0
    model.eval()
    all_predictions = torch.tensor([])
    all_targets = torch.tensor([])
    for batch in valid_dl:
        inputs, targets = batch
        inputs = inputs.long().to(DEVICE)
        targets = targets.to(DEVICE)

        with torch.no_grad():
            output = model(inputs)

        predictions = output.argmax(1)
        all_targets = torch.cat([all_targets, targets.detach().cpu()])
        all_predictions = torch.cat([all_predictions, predictions.detach().cpu()])
        
    mean_loss /= len(valid_dl)
    val_acc = (all_predictions == all_targets).float().mean().numpy()
    print("Validation accuracy:",val_acc)
    print("Mean Loss:", mean_loss)
    if val_acc > best_val_acc:
        torch.save(model.state_dict(), "./model")
        best_val_acc = val_acc

print("Best Validation accuracy:", best_val_acc)
print("Final Mean Loss:", mean_loss)

Epoch #1
Validation accuracy: 0.8862
Mean Loss: 0.0
Epoch #2
Validation accuracy: 0.883
Mean Loss: 0.0
Epoch #3
Validation accuracy: 0.8578
Mean Loss: 0.0
Epoch #4
Validation accuracy: 0.9008
Mean Loss: 0.0
Epoch #5
Validation accuracy: 0.8986
Mean Loss: 0.0
Epoch #6
Validation accuracy: 0.9052
Mean Loss: 0.0
Epoch #7
Validation accuracy: 0.9056
Mean Loss: 0.0
Epoch #8
Validation accuracy: 0.9016
Mean Loss: 0.0
Epoch #9
Validation accuracy: 0.9046
Mean Loss: 0.0
Epoch #10
Validation accuracy: 0.903
Mean Loss: 0.0
Best Validation accuracy: 0.9056
Final Mean Loss: 0.0


In [80]:
model.load_state_dict(torch.load("./model_28Mar"))

<All keys matched successfully>

In [81]:
model.eval()
all_predictions = torch.tensor([])
all_targets = torch.tensor([])
#print(test_df)
for batch in test_dl:
    inputs, targets = batch
    inputs = inputs.long().to(DEVICE)
    targets = targets.to(DEVICE)

    with torch.no_grad():
        output = model(inputs)

    predictions = output.argmax(1)
    #print(predictions)
    all_targets = torch.cat([all_targets, targets.detach().cpu()])
    all_predictions = torch.cat([all_predictions, predictions.detach().cpu()])

val_acc = (all_predictions == all_targets).float().mean().numpy()
print("Test Accuracy:", val_acc)

Test Accuracy: 0.909


In [93]:
ex_text_str = 'If you love Spider-Man then you will definitely love this film. Not only does "Spider-Man: No Way Home" have an amazing script but the characters are written perfectly. Not only will this blockbuster blow your socks off visually and emotionally but it will bring back a lot of needed nostalgia that is a breath of fresh air for Spider-Man fans of all generations.'

d = {'review': [ex_text_str]}
X_test = pd.DataFrame(data=d)
X_test.review = df.review.apply(lower)
X_test.review = df.review.apply(deleteHtmlTags)
X_test.review = df.review.apply(unidecode)
X_test.review = df.review.apply(deletePunctuation)
X_test.review = df.review.apply(unescape)
X_test.review = df.review.apply(deleteSpaces)
X_test["stemmed"] = X_test.review.apply(lambda x: [stemmer.stem(y) for y in x.split() if stemmer.stem(y) not in (stop)])
X_test_vectorized=vectorize_sentences(X_test.stemmed, token_indices)
X_test_vectorized = pad(X_test_vectorized, max_length = max_len)
X_test_ds = Dataset(X_test_vectorized,[1])
X_test_dl = torch.utils.data.DataLoader(test_ds, batch_size=1, shuffle=False)
model.eval()
all_predictions = torch.tensor([])
all_targets = torch.tensor([])
i=0
#print(test_df)
for batch in X_test_dl:
    #print(i)
    i=i+1
    inputs, targets = batch
    inputs = inputs.long().to(DEVICE)
    targets = targets.to(DEVICE)
    #print("targets",targets)
    with torch.no_grad():
        output = model(inputs)

    predictions = output.argmax(1)
    
    all_targets = torch.cat([all_targets, targets.detach().cpu()])
    all_predictions = torch.cat([all_predictions, predictions.detach().cpu()])

val_acc = (all_predictions == all_targets).float().mean().numpy()
print("Test Accuracy:", val_acc)
print(len(all_predictions))
print(len(all_targets))
print(all_predictions==all_targets)


Test Accuracy: 0.4962
5000
5000
tensor([False, False,  True,  ...,  True, False,  True])
